# Purpose: Use dimensions.ai as the source of summary statistics of research publication by category, try to identify categories with highest growth (hence more potential)
last updated on 3/17/2023
1. Collect research cateogies and codes from https://app.dimensions.ai/browse/categories/publication/for
2. Visit each catogory and crawl research data
3. Analyze growth trend and compare across time and different sectors

References: 
1. https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a

In [ ]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \AppleWebKit/537.36 (KHTML, like Gecko) \Chrome/90.0.4430.212 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})

In [ ]:
def get_soup(url,HEADERS):
    r = requests.get(url, headers=HEADERS).text
#     soup = BeautifulSoup(r, 'html.parser')
    soup = BeautifulSoup(r, 'lxml')
    return soup
soup= get_soup("https://app.dimensions.ai/browse/categories/publication/for",HEADERS)

In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

soup = BeautifulSoup(urlopen("https://app.dimensions.ai/browse/categories/publication/for"))
result = soup.find_all("div", {"class":"mycategories"})

for res in result:
    print(res.decode_contents().strip())

Note: used selenium instead of BeautifulSoup because the category data was loaded after loading the site

In [ ]:
# use the latest selenium 4, there are syntax changes from previous version
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
driver.get("https://app.dimensions.ai/browse/categories/publication/for")

In [ ]:
all_href = driver.find_elements(By.XPATH,  "//*[contains(@href, '/discover/publication?and_facet_for')]")

In [ ]:
# Find by div class failed because can't collect the href links
# level_1 = driver.find_elements("xpath",'//div[@class="sc-2f5ewc-3 bVEbpe textcrop"]')
# level_2 = driver.find_elements("xpath",'//div[@class="sc-2f5ewc-0 gIuuzb textcrop"]')

In [ ]:
research_collection = {}
for i in range(len(all_href)):
    research_collection[all_href[i].text] = all_href[i].get_attribute('href')

In [ ]:
research_data = {}
for item in list(research_collection.items()):
    name, link = item[0],item[1]
    driver.get(link.replace("discover/publication?","analytics/publication/overview/timeline?"))
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='analytics_infinite_scroll_container']/div[2]/div[2]/div[2]/div[2]/fieldset/span[3]"))).click()
    WebDriverWait(driver, 10)
    # Reference: https://www.softwaretestingmaterial.com/how-to-handle-web-tables-in-selenium-python/
    columns_header = driver.find_elements(By.XPATH,"//table[@class= 'table table--analytical']/thead/tr/th")
#     row_header = driver.find_elements(By.XPATH,"//table[@class= 'table table--analytical']/tbody/tr/th")
    row_data = driver.find_elements(By.XPATH,"//table[@class= 'table table--analytical']/tbody/tr/td")
    data = {}
    for i in range(len(columns_header)):
        data[columns_header[i].text] = row_data[i].text
    research_data[name] = data

In [ ]:
# Reference: https://pynative.com/python-save-dictionary-to-file/
import pickle
with open('research_data.pkl', 'wb') as fp:
    pickle.dump(research_data, fp)

In [ ]:
with open('research_data.pkl', 'rb') as fp:
    research_data = pickle.load(fp)

In [ ]:
driver.close()

In [ ]:
research_data